# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.widgets import RunDetails

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [23]:
ws = Workspace.from_config()
experiment_name = 'hyper-drive-experiment'

experiment=Experiment(ws, experiment_name)

datastore = ws.get_default_datastore()
datastore.upload_files(files = ['../cleandata.csv'],
                    target_path = 'train-dataset/tabular/',
                    overwrite = True,
                    show_progress = True)

Uploading an estimated of 1 files
Uploading ../cleandata.csv
Uploaded ../cleandata.csv, 1 files out of an estimated total of 1
Uploaded 1 files


$AZUREML_DATAREFERENCE_7c1033344e7c45589b747a31720276a6

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute

cluster_name = "cpuCluster"
compute_config = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2",max_nodes=4)
cpu_cluster = ComputeTarget.create(ws,cluster_name,compute_config)
cpu_cluster.wait_for_completion(show_output=True)

InProgress.
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [27]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(slack_factor=0.1, evaluation_interval=1,delay_evaluation=5)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling({
    "C": uniform(0.05,0.1),
    "max_iter": choice(16,32,64,128)
    })

env = Environment.from_conda_specification(name='sklearn-env', file_path="depend.yml")

#TODO: Create your estimator and hyperdrive config
estimator = ScriptRunConfig(source_directory="./",script="train-dataset.py", compute_target=cluster_name, environment=env)

hyperdrive_run_config = HyperDriveConfig(
    run_config=estimator,
    hyperparameter_sampling=param_sampling,
    policy=early_termination_policy,
    primary_metric_name="Accuracy",
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=20
)

In [28]:
#TODO: Submit your experiment
run = experiment.submit(config=hyperdrive_run_config, show_output=True)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [29]:
RunDetails(run).show

<bound method _WidgetRunDetailsBase.show of <azureml.widgets._hyperdrive._run_details._HyperDriveRunDetails object at 0x7f5cf2acf8b0>>

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
#TODO: Save the best model

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

